In [1]:
import numpy as np
import random

In [29]:
import networkx as nx
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\ENZYMES_g102.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\aves-sparrowlyon-flock-season2-unw.edges", nodetype=int)
#G = nx.read_edgelist("brock200_1_complement.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\sparrow_unweighted.edges", nodetype=int)
G = nx.read_edgelist(r"..\vertex_cover_benchmarks\cats.edges", nodetype=int)

In [5]:
values = []
weights = []

with open(r"..\knapsack_benchmarks\s000.kp") as f:
    n, W = [int(x) for x in next(f).split()] # read first line
    for line in f: # read rest of lines
        nums = line.split()
        values.append(int(nums[0]))
        weights.append(int(nums[1]))

sum_values = sum(values)
print(values)
print(weights)
import random

[845, 758, 421, 259, 512, 405, 784, 304, 477, 584, 909, 505, 282, 756, 619, 251, 910, 983, 811, 903, 311, 730, 899, 684, 473, 101, 435, 611, 914, 967, 478, 866, 261, 806, 549, 15, 720, 399, 825, 669, 2, 494, 868, 244, 326, 871, 192, 568, 239, 968]
[804, 448, 81, 321, 508, 933, 110, 552, 707, 548, 815, 541, 964, 604, 588, 445, 597, 385, 576, 291, 190, 187, 613, 657, 477, 90, 758, 877, 924, 843, 899, 924, 541, 392, 706, 276, 812, 850, 896, 590, 950, 580, 451, 661, 997, 917, 794, 83, 613, 487]


In [29]:
#MAX_FITNESS = G.order() # Tal y como está definido, el máximo valor que puede tomar el fitness es el número de nodos del grafo
MAX_FITNESS = 1
MAX_ITER = 5000
SAMPLE_SIZE = 100
INITIAL_SAMPLE_SIZE = 300

In [4]:
class Solution:
    def __init__(self, sol, f = 0):
        self.sol = sol
        self.f = f

    def set_f(self, f):
        self.f = f

In [32]:
def subsets(a):
  l = len(a)
  n = l - 1
  x = [0] * n
  f = [j for j in range(l)]
  r = list()
  
  while n:
    m = [0] + x # insert 0 at the beginning of the bit string forms
    subset = [a[i] for i in range(l) if m[i] == 1] # generate the subset
    
    # and use the subset to generate the subsets for when 1 is inserted at the beginning
    k = [a[0]] + subset
    
    r.append(subset) # add to the table all subsets of bits strigs with prefix 0
    r.append(k) # add to the table all subsets of bits strings with prefix 1
    
    # initialization
    # this is where we choose which j we want to change
    j = f[0]
    f[0] = 0
    
    # if j == n we terminate because we have all the possible bit strings
    if j == n:
      break
     
    # update our array when j != n
    f[j] = f[j + 1]
    f[j + 1] = j + 1
    
    x[j] = 1 - x[j] # complement coordinate j and return to line 8 if j != n
  
  return r

In [33]:
def is_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            return False
    return True

In [34]:
def naive_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            node = random.choice([u, v])
            sol[node - 1] = 1
    return

In [35]:
def fitness(sol):
    print("oye por favor")
    n = G.order()
    if is_cover(sol, G):
        return n - sum(sol)
    else:
        if sum(sol) >= (2 * n // 3):
            print("estoy en sitio bueno")
            zeros_index = []
            for i in range(len(sol)):
                if sol[i] == 0:
                    zeros_index.append(i)
                    #sol[i] = 1
                    #if is_cover(sol, G):
                    #    print(sol)
                    #    break
                    #sol[i] = 0
            r = subsets(zeros_index)
            r.sort(key=len)
            # r[0] es el conjunto vacío
            for subset in r[1:]:
                for index in subset:
                    sol[index] = 1 # Esto modifica sol porque se pasa por referencia
                if is_cover(sol, G):
                    print(f"found fitness {n - sum(sol)}")
                    return n - sum(sol)
                for index in subset:
                    sol[index] = 0
            #print("calavera 1")
            return 0
        else:
            naive_cover(sol, G)
            #print("calavera 2")
            return n - sum(sol)

In [36]:
(G.order() - 10) / G.order()

0.8461538461538461

In [37]:
def fitness2(sol):
    if is_cover(sol, G):
        print("found")
        n = G.order()
        return (n - sum(sol)) / n
    else:
        return 0

In [38]:
def fitness3(sol):
    n = G.order()
    if is_cover(sol, G):
        #print("buenísimo")
        return (n - sum(sol)) / sum(sol)
    else:
        if sum(sol) >= (2 * n // 3):
            #print("estoy en sitio bueno")
            zeros_index = []
            for i in range(len(sol)):
                if sol[i] == 0:
                    zeros_index.append(i)
                    #sol[i] = 1
                    #if is_cover(sol, G):
                    #    print(sol)
                    #    break
                    #sol[i] = 0
            r = subsets(zeros_index)
            r.sort(key=len)
            # r[0] es el conjunto vacío
            for subset in r[1:]:
                for index in subset:
                    sol[index] = 1 # Esto modifica sol porque se pasa por referencia
                if is_cover(sol, G):
                    #print(f"found fitness {(n - sum(sol)) / sum(sol)}")
                    return (n - sum(sol)) / sum(sol)
                for index in subset:
                    sol[index] = 0
            assert(False)
            naive_cover(sol, G) # Pensándolo, aquí no debería llegar porque siempre se puede completar
            print("calavera 1")
            return (n - sum(sol)) / sum(sol)
        else:
            naive_cover(sol, G)
            #print("calavera 2")
            return (n - sum(sol)) / sum(sol)

In [39]:
def fitness4(sol):
    n = G.order()
    if not is_cover(sol, G):
        naive_cover(sol, G)
    return (n - sum(sol)) / sum(sol)

In [6]:
def fitness_mochila(sol):
    n = len(sol)
    fit = 0
    wei = 0
    for i in range(n):
        fit += values[i] * sol[i]
        wei += weights[i] * sol[i]
    while wei > W:
        sacar = random.randrange(0, n)
        wei -= weights[sacar] * sol[sacar]
        fit -= values[sacar] * sol[sacar]
        sol[sacar] = 0
    return fit / sum_values

In [7]:
def f(sol):
    fit = fitness_mochila(sol.sol)
    sol.set_f(fit)
    #print(f"FITNESS de {sol.sol} = {fit}")
    return fit

In [8]:
def hamming_distance(sol1, sol2):
    n = len(sol1)
    #print(f"sol1 = {sol1}\nsol2 = {sol2}")
    assert(n == len(sol2))
    return [sol1[i] == sol2[i] for i in range(n)].count(False) ** 3

In [9]:
def inverse_distance(sol1, sol2, dist):
    d = dist(sol1, sol2)
    return 1 / d if d != 0 else 2 # Realmente la forma de manejar el infinito depende de la distancia
# El 2 funciona para Hammings porque no puede haber distancias d 0 < d < 1 (es decir, tales que 1 / d > 2).

In [43]:
inverse_distance([1,0,1,0], [0,1,0,1], hamming_distance)

0.015625

In [10]:
def w(sol1, sol2, A, dist):
    assert(len(sol1) == len(sol2))
    sum = 0
    for point in A:
        sol = point.sol
        assert(len(sol) == len(sol1))
        sum += inverse_distance(sol1, sol, dist)
    assert(sum > 0) # Creo que se puede demostrar que esto no va a ser 0.
    inv = inverse_distance(sol1, sol2, dist)
    res = inv / sum
    #print(f"inv: {inv}\nsum: {sum}\nres: {res}")
    return res

In [11]:
def g(sol, A, dist):
    sum = 0
    for point in A:
        wei = w(sol.sol, point.sol, A, dist)
        fit = point.f
        #print(f"\nwei: {wei} \n fit: {fit}")
        sum += wei * fit
    #print(f"\n\n g: {sum}\n\n")
    return sum

In [12]:
def search_optimal(A):
    card = len(A)
    assert(card > 0)
    opt = A[0]
    max_fit = opt.f
    for elem in A[1:]:
        fit = elem.f
        if fit > max_fit:
            max_fit = fit
            opt = elem
    return opt

In [13]:
def search_nearest(sol, A, dist):
    card = len(A)
    assert(card > 0)
    nearest = A[0]
    min_distance = dist(sol.sol, nearest.sol)
    for elem in A[1:]:
        distance = dist(sol.sol, elem.sol)
        if distance < min_distance:
            min_distance = distance
            nearest = elem
    return nearest

In [14]:
def search_2_nearest(sol, A, dist):
    card = len(A)
    assert(card > 1)
    first_distance = dist(sol.sol, A[0].sol)
    second_distance = dist(sol.sol, A[1].sol)
    if first_distance > second_distance:
        nearest = A[1]
        second = A[0]
        min_distance = second_distance
        sec_min_distance = first_distance
    else:
        nearest = A[0]
        second = A[1]
        min_distance = first_distance
        sec_min_distance = second_distance
    for elem in A[2:]:
        distance = dist(sol.sol, elem.sol)
        if distance < sec_min_distance:
            if distance < min_distance:
                sec_min_distance = min_distance
                min_distance = distance
                second = nearest
                nearest = elem
            else:
                sec_min_distance = distance
                second = elem
    return [nearest, second]

In [27]:
def sample(k, A, dist, dimension, skip = []):
    sample = []
    while len(sample) < k:
        x = [random.choice([0, 1]) for _ in range(dimension)]
        if x in sample or x in skip:
            continue
        #y = random.randint(0, MAX_FITNESS)
        y = random.uniform(0, MAX_FITNESS)
        sol = Solution(x)
        set_points = [search_optimal(A)] + search_2_nearest(sol, A, dist)
        g_val = g(sol, set_points, dist)
        if g_val > y:
            #print(fitness(x))
            f(sol)
            #print(f"Muestra añadida {x}\ncon valor de g: {g_val} y valor de y: {y}")
            sample.append(sol)
    return sample

In [22]:
def init(k, dimension):
    init = []
    while len(init) < k:
        x = [random.choice([0, 1]) for _ in range(dimension)]
        if not any(point.sol == x for point in init): # Esto se evita usando un set. Aún no tengo claro si necesito índices, o si el set es eficiente en python.
            sol = Solution(x)
            f(sol)
            init.append(sol)
    return init

def max_k_fitness(set_sols, k):
    sor = sorted([(fitness(elem), elem) for elem in set_sols])
    return [elem for (_,elem) in sor][-k:]

def max_fitness(set_sols):
    max_value = fitness(set_sols[0])
    max_elem = set_sols[0]
    for elem in set_sols[1:]:
        f = fitness(elem)
        if f > max_value:
            max_value = f
            max_elem = elem
    return max_elem, max_value

def next_gen(curr_gen, k):
    new = sample(k, skip = curr_gen)
    return max_k_fitness(curr_gen + new, k)

def algorithm():
    gen = init(SAMPLE_SIZE)
    for _ in range(MAX_ITER):
        gen = next_gen(gen, SAMPLE_SIZE)
    return max_fitness(gen)

In [17]:
def max_f(A):
    assert(len(A) > 0)
    max_value = A[0].f
    max_elem = A[0]
    for elem in A[1:]:
        fit = elem.f
        if fit > max_value:
            max_value = fit
            max_elem = elem
    return max_elem, max_value

In [18]:
def alg(sample_size, num_iter, initial_sample_size, dist):
    #dic.clear()
    A = init(initial_sample_size)
    for i in range(num_iter):
        print(f"iter {i}")
        A.extend(sample(sample_size, A, dist, skip = A))
    return max_f(A)

In [42]:
def next_swarm_step(k, A, dist, dimension):
    opt = search_optimal(A)
    next = sample(k - 1, A, dist, dimension, skip = A)
    next.append(opt)
    return next

In [23]:
def alg2(sample_size, num_iter, dist, dimension):
    A = init(sample_size, dimension)
    for i in range(num_iter):
        A = next_swarm_step(sample_size, A, dist, dimension)
    return max_f(A)

In [55]:
"""
elem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance)
print(f'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con {sum(elem.sol)} nodos.')

print(is_cover(elem.sol, G))

num_ceros = G.order() - sum(elem.sol)
print(num_ceros, "ceros")
"""

'\nelem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance)\nprint(f\'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con {sum(elem.sol)} nodos.\')\n\nprint(is_cover(elem.sol, G))\n\nnum_ceros = G.order() - sum(elem.sol)\nprint(num_ceros, "ceros")\n'

In [56]:
#for i in range(20):
#    elem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance)
#    print(f'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con {sum(elem.sol)} nodos.')

#    print(is_cover(elem.sol, G))

#    num_ceros = G.order() - sum(elem.sol)
#    print(num_ceros, "ceros")

In [44]:
output = open(r"..\results\kn0\medias_ponderadas_hib.csv", "a")
for i in range(20):
    print(f"ejec {i}")
    elem, _ = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dimension = n, dist = hamming_distance)
    output.write(f'{elem.sol},')
    fit = 0
    for i in range(n):
        fit += values[i] * elem.sol[i]
    #num_ceros = G.order() - sum(elem.sol)
    print(fit)
    output.write(f'{fit}\n')
output.close()

ejec 0
19076
ejec 1
18888
ejec 2
19003
ejec 3
19220
ejec 4
18993
ejec 5
19693
ejec 6
18798
ejec 7
19239
ejec 8
19213
ejec 9
18923
ejec 10
18896
ejec 11
19291
ejec 12
19113
ejec 13
19147
ejec 14
19202
ejec 15
19001
ejec 16
18897
ejec 17
19485
ejec 18
19262
ejec 19
19018
